# **Householder Notebook**

Maby by eti.calde

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

## **1.1 Householder Reflection**

**Definition**
Given a unit vector $u$ defining a hyperplane perpendicular to ir, a Householder reflection over a vector $x$ is defined as:

$$
H_{\mathbf{u}}(x) = x-2 \langle \mathbf{x},\mathbf{u}\rangle \mathbf{u}
$$

Is nothing more that the vector reflected over the hyperplane. This can be expressed in matrix form as follows:

$$
H =(I-2\mathbf{u}^T\mathbf{u})
$$

---

## **1.2 let’s see the transformation in code!!**

In [2]:
def get_householder_matrix(v):
    """
    Calculates the Householder reflection matrix for a vector v
    
    Parameters
    ----------
    v : ndarray (n,), input vector.
    Returns
    -------
    H : ndarray (n,n), Householder reflection matrix.
    """
    v = v.reshape(-1, 1)
    I = np.eye(v.shape[0])
    return I - 2 * (v @ v.T) / (v.T @ v)

In [34]:
cmap = plt.get_cmap("magma")
C_V1   = cmap(0.15)  # x
C_V2   = cmap(0.35)  # u
C_PLN  = cmap(0.85)  # hyperplane
C_REF  = cmap(0.60)  # H(x)

def plot_householder(theta1_deg=30.0, theta2_deg=120.0, r1=1.5):
    # Parameters
    t1 = np.deg2rad(theta1_deg)
    t2 = np.deg2rad(theta2_deg)
    unitari_rad = 1.0

    # Vectors
    x = np.array([r1*np.cos(t1), r1*np.sin(t1)])
    u = np.array([unitari_rad*np.cos(t2), unitari_rad*np.sin(t2)])

    # Hyperplane
    n_perp = np.array([-u[1], u[0]])

    # Householder
    H = get_householder_matrix(u)
    x_householder = H @ x
    print("x:", x)
    print("x' (Householder):", x_householder)
    # ===================================================================
    # PLOT
    plt.figure()

    # Vectors
    plt.quiver(0, 0, x[0], x[1], color=C_V1, scale=1, scale_units='xy', angles='xy', label='x')
    plt.quiver(0, 0, u[0], u[1], color=C_V2, scale=1, scale_units='xy', angles='xy', label='u (unitario)')
    plt.quiver(0, 0, x_householder[0], x_householder[1],
               color=C_REF, scale=1, scale_units='xy', angles='xy', label='H(x)')
    # Hyperplane
    L = 2.2
    pts = np.vstack([-L*n_perp, L*n_perp])
    plt.plot(pts[:,0], pts[:,1], linestyle='--', color=C_PLN, label='Hip. u·x = 0')

    # Details
    lim = 2.2
    plt.xlim(-lim, lim); plt.ylim(-lim, lim)
    plt.grid(True)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.title("Householder reflection of x")
    plt.show()

angle1 = FloatSlider(value=30.0,  min=0.0, max=360.0, step=1.0, description='Ángulo x (°)')
angle2 = FloatSlider(value=120.0, min=0.0, max=360.0, step=1.0, description='Ángulo u (°)')
len_r1 = FloatSlider(value=1.5,   min=0.0, max=2.0,   step=0.1, description='||x||')

interact(plot_householder, theta1_deg=angle1, theta2_deg=angle2, r1=len_r1)


interactive(children=(FloatSlider(value=30.0, description='Ángulo x (°)', max=360.0, step=1.0), FloatSlider(va…

<function __main__.plot_householder(theta1_deg=30.0, theta2_deg=120.0, r1=1.5)>

Now let's apply the Householder Reflection to get the QR factorization of a matrix $A$.

---

## **2.2 Get a random matrix**

In [4]:
def _random_orthogonal(n, rng):
    """Matriz ortogonal ~ Haar usando QR sobre una matriz gaussiana."""
    M = rng.standard_normal((n, n))
    Q, R = np.linalg.qr(M)
    # Asegurar unicidad de signos (evita que R tenga diag negativa)
    d = np.sign(np.diag(R))
    d[d == 0] = 1.0
    Q = Q * d
    return Q

def make_matrix_with_condition(n, kappa=10.0, structure="general", spectrum="geom", seed=None):
    """
    Crea una matriz A de tamaño n×n con número de condición ~kappa en norma 2.
    structure: "general" (no simétrica) o "spd" (simétrica definida positiva).
    spectrum:  "geom" reparte singular/propios en progresión geométrica entre [1/kappa, 1].
    """
    if kappa < 1:
        raise ValueError("kappa debe ser >= 1")
    rng = np.random.default_rng(seed)

    # Valores singulares/propios: de 1/kappa (mín) a 1 (máx)
    if spectrum == "geom":
        vals = np.geomspace(1.0, 1.0/kappa, num=n)  # descendente; max=1, min=1/kappa
        vals = np.sort(vals)[::-1]                  # garantizar orden descendente
    else:
        raise ValueError("spectrum no soportado (usa 'geom')")

    if structure == "general":
        U = _random_orthogonal(n, rng)
        V = _random_orthogonal(n, rng)
        A = U @ np.diag(vals) @ V.T
    elif structure == "spd":
        Q = _random_orthogonal(n, rng)
        A = Q @ np.diag(vals) @ Q.T   # simétrica y definida positiva
    else:
        raise ValueError("structure debe ser 'general' o 'spd'")

    # Reescala opcional: no cambia kappa; útil si quieres ||A|| ~ 1, etc.
    # A = A / np.linalg.norm(A, 2)

    # Verificación empírica (puede diferir ~1e-12 por redondeo)
    s = np.linalg.svd(A, compute_uv=False)
    kappa_emp = s[0] / s[-1]
    return A, kappa_emp

def generate_system(N, kappa=10.0, structure="general", seed=None, x_known=None):
    """
    Genera (A, x_true, b) con cond_2(A) ~ kappa.
    - N: tamaño
    - structure: 'general' o 'spd'
    - x_known: si lo pasas, se usa; si no, se crea aleatorio.
    """
    rng = np.random.default_rng(seed)
    A, kappa_emp = make_matrix_with_condition(N, kappa=kappa, structure=structure, seed=seed)
    if x_known is None:
        x_true = rng.standard_normal(N)
    else:
        x_true = np.asarray(x_known, dtype=float)
        if x_true.shape != (N,):
            raise ValueError("x_known debe tener shape (N,)")
    b = A @ x_true
    return A, x_true, b, kappa_emp


In [5]:
N = 5
kappa_deseada = 100
A, x_true, b, kappa_emp = generate_system(N, kappa=kappa_deseada, structure="general", seed=42)

print("condición objetivo:", kappa_deseada)
print("condición empírica:", kappa_emp)

x_rec = np.linalg.lstsq(A, b, rcond=None)[0]
print("error relativo en x:", np.linalg.norm(x_rec - x_true) / np.linalg.norm(x_true))
print("A =\n", A)
print("x_true =\n", x_true)
print("b =\n", b)

condición objetivo: 100
condición empírica: 100.00000000000038
error relativo en x: 3.0865966542260165e-15
A =
 [[-0.13081689  0.16657923  0.07723457 -0.03782061 -0.15396884]
 [ 0.11084154 -0.58528037 -0.33288669 -0.20974112 -0.0265564 ]
 [ 0.00141675  0.39448968  0.17665277  0.20730293  0.17369555]
 [ 0.08366047 -0.41891365 -0.20390799 -0.0688255   0.00174923]
 [-0.02311784 -0.02242665 -0.08526961 -0.13976344 -0.09035459]]
x_true =
 [ 0.30471708 -1.03998411  0.7504512   0.94056472 -1.95103519]
b =
 [ 0.10968479  0.24717976 -0.42116632  0.23998568 -0.00288329]


## **2.3 QR Factorization**

In [ ]:
def get_Q_Matrix(householder_vectors):
    """
    Calculates Q matrix of QR factorization using Householder reflections.

    Parameters
    ----------
    householder_vectors : ndarray (n,n), Householder reflection vectors.
    each i-th column represents a Householder vector for the i-th step of the QR factorization.

    Returns
    -------
    Q : ndarray (n,n), Orthogonal matrix.
    """
    n = householder_vectors.shape[0]
    Q = np.eye(n)
    for i in range(n):
        u = householder_vectors[:n-i, i]
        H_tilde = get_householder_matrix(u)
        H_full = np.eye(n)
        H_full[i:, i:] = H_tilde
        Q = Q @ H_full
    return Q

def QR_Householder(A):
    """
    QR factorization using Householder reflections.

    Parameters
    ----------
    A : ndarray (n,n), input matrix.
    Returns
    -------
    Q,R : ndarray (n,n), Orthogonal matrix and upper triangular matrix.
    """
    if (A.shape[1] != A.shape[0]):
        print("Matrix is not square")
        return None, None
    
    n = A.shape[0]
    # We need to work with submatrices of A
    # and we don't want to modify the original Matrix
    A = A.copy()

    # Each Householder vector is 1 dimension smaller than the previous one,
    # So we use just the first n-i rows on each iteration 
    householder_vectors = np.zeros((n, n))

    for i in range(n):
        sub_A = A[i:, i:]
        # Get Column i of A
        col_i_of_A = sub_A[:,0]
        col_i_of_A = col_i_of_A.reshape(-1,1)

        # Compute the Householder vector
        alpha = -np.sign(col_i_of_A[0,0]) * np.linalg.norm(col_i_of_A)
        e1 = np.eye(col_i_of_A.shape[0])[:,0].reshape(-1,1)
        u = col_i_of_A - alpha * e1
        u = u / np.linalg.norm(u)
        # print(f"u en la iteracion {i}, {u.shape} =\n", u)

        # Save the Householder vector
        householder_vectors[0:n-i,i] = u.ravel()
        # print(f"holder_vectors en la iteracion {i} =\n", holder_vectors)
        
        # FIX RESPECTO A LA VERSIÓN DE LA CALSE ANTERIOR.
        A[i:, i:] -= 2 * np.outer(u, u.T @ A[i:, i:])

    Q = get_Q_Matrix(householder_vectors)
    R = np.triu(A)
    # R = A
    return Q,R

Q,R = QR_Householder(A)
# print("uni?:\n", Q.T@Q)
compare_numpy = True
if (N <= 5):
    # print("A =\n", A)
    print("R:\n", R)
    print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A - Q @ R))
    if compare_numpy:
        A_np, Q_np, R_np = A.copy(), Q.copy(), R.copy()
        print("Comparando con numpy...")
        Q_np, R_np = np.linalg.qr(A_np)
        print("R (numpy):\n", R_np)
        print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A_np - Q_np @ R_np))
else:
    print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A - Q @ R))
    if compare_numpy:
        A_np, Q_np, R_np = A.copy(), Q.copy(), R.copy()
        print("Comparando con numpy...")
        Q_np, R_np = np.linalg.qr(A_np)
        print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A_np - Q_np @ R_np))

uni?:
 [[ 1.00000000e+00 -6.93889390e-17 -2.77555756e-17]
 [-6.93889390e-17  1.00000000e+00  0.00000000e+00]
 [-2.77555756e-17  0.00000000e+00  1.00000000e+00]]
R:
 [[ 0.71093096 -0.0235202   0.68880925]
 [ 0.         -0.30696658  0.11974985]
 [ 0.          0.          0.14490437]]
Reconstrucción ||A - Q R|| = 3.164812698569833e-16
Comparando con numpy...
R (numpy):
 [[ 0.71093096 -0.0235202   0.68880925]
 [ 0.         -0.30696658  0.11974985]
 [ 0.          0.         -0.14490437]]
Reconstrucción ||A - Q R|| = 4.690302206656113e-16


## **2.4 Arnoldi Householder**
![image.png](pseudocodigo-householder-arnorli.png)


### **1. seleccionar un v non zero**
$$
z_1 = v = \frac{r_0}{\| r_0 \|_2}
$$

### **2. Loop `j desde 1,2,...,m,m+1`**

### **3 y 4. Cálcular el vector de householder $w_j$,** tal que:
$$
¿w_j = A v_j ?
$$
Del cual sabemos que tiene los primeros $j-1$ elementos nulos
$$
(w_j) = 0 \quad i = 1,\dots, j-1 \\
$$
### **5. definir la transformada de Householder**
$$
P_j = I - 2 w_j w_j^T
$$
Y la transformación tiene que cumplir que 
$$
P_j z_j = 0 \quad i = j+1,\dots,n
$$

### **6. actualizar $h$**
$$
h_{j-1} = P_j z_j
$$
los primeros $j$ coeficientes son los coeficientes de la matriz de Hessenberg y desde $j+1$ ceros, por como se definió $P_j z_j$.

### **7. Actualizar $v_j$**
$$
v_j = P_1 P_2 \dots P_j e_j
$$
aplicamos el producto acumulado de reflectores el j-ésimo vector canónico. según Lema 2.1 es el siguiente vector de la base.

### **6. actualizar $z_{j+1}$**
$$
z_{j+1} = P_j P_{j-1} \dots P1 Av_j
$$
Explicación Lema 2.1

In [ ]:
def getP1(v):
    v = np.copy(v)
    signo = np.copysign(1.0, v[0])
    norm_v = np.linalg.norm(v)
    e1_escalado = np.zeros_like(v)
    e1_escalado[0] = 1
  
    u = v - (signo * norm_v * e1_escalado)
    u = u / np.linalg.norm(u) 
    return u    

def apply_householder(u,v):
    """
    Aplica una transformación de Householder definida por 'u' a un vector 'v'.
    """    
    I = np.eye(len(v))
    P = I - 2 * np.outer(u, u)
    return P @ v

def get_canonical_vec(size, index):
    vec = np.zeros(size)
    vec[index] = 1
    return vec

def get_householder_vector(v, start_index):
    """
    Calcula el vector de Householder 'u' para anular los elementos de v
    desde start_index + 1 en adelante.
    """
    n = len(v)
    
    s = v[start_index:]
    
    norm_s = np.linalg.norm(s)
        
    target = np.zeros_like(s)
    signo = np.copysign(1.0, s[0])
    target[0] = signo * norm_s
    
    u_s = s - target
    norm_u_s = np.linalg.norm(u_s)
    u_s = u_s / norm_u_s
    
    u_full = np.zeros(n)
    u_full[start_index:] = u_s
    
    return u_full

def arnoldi_walker(A,b):
    n = A.shape[0]
    H = np.zeros((n, n))
    V = np.zeros((n, n))

    # INITIAL SETUP
    # 1). v1
    v1 = b / np.linalg.norm(b)
    
    # 1). P1 tal que P1 v = e1
    P1 = getP1(v1)

    householder_reflections = [P1]
    # 2). Main Loop
    for j in range(n):
        # a). v_j = P1 ... Pj e_j
        e_j = np.zeros(n)
        e_j[j] = 1.0
        v_j = e_j
        for P in reversed(householder_reflections):
            v_j = apply_householder(P, v_j)
        V[:, j] = v_j

        # b). Choose P_{m+1} such that P_{m+1} ... P1 (v1, Av1,...,Avm) is upper triangular
        P_j_plus_1 = get_householder_vector(A @ v_j, start_index=j)
        householder_reflections.append(P_j_plus_1)

    P1 = householder_reflections[0]
    v1 = V[:, 0]
    print("P1 @ V1:", apply_householder(P1, v1))

    P2 = householder_reflections[1]
    v2 = V[:, 1]
    print("P2 @ V2:", apply_householder(P2, v2))

    P3 = householder_reflections[2]
    v3 = V[:, 2]
    print("P3 @ V3:", apply_householder(P3, v3))

    return V, H

N = 3
kappa_deseada = 10
A, x_true, b, kappa_emp = generate_system(N, kappa=kappa_deseada, structure="general", seed=42)

V, H = arnoldi_walker(A, b)

print("Matriz A:\n", A)
print("\n Matriz de Hessenberg H:\n", H)
print("\n Matriz de Krylov V (ortogonal):\n", V)


# random_vec_5 = np.ones(5)
# u5 = get_householder_vector(random_vec_5,1)
# print("print u5:", random_vec_5)
# print("house holder vec",u5)
# print("Householder debería eliminar a partir del elmento 1+1:\n", apply_householder(u5, random_vec_5))


print("\nVerificando que V es ortogonal...")
ortogonal = np.allclose(V.T @ V, np.eye(N))
print(f"¿Es V ortogonal? {'Sí :D' if ortogonal else 'No :c'}")

# print("\nVerificando la relación AV = VH...")
# AV = A @ V
# VH = V @ H

# print("\nResultado de A @ V:\n", AV)
# print("\nResultado de V @ H:\n", VH)

# are_close = np.allclose(AV, VH)
# print(f"\n¿Son AV y VH suficientemente cercanos? {'Sí :D' if are_close else 'No :c'}")

check P1 v1: [ 1.00000000e+00 -3.13984949e-16 -2.77555756e-17]
P1 @ V1: [ 1.00000000e+00 -3.83373888e-16 -5.55111512e-17]
P2 @ V2: [0.34712522 0.01001533 0.93776531]
P3 @ V3: [0.78670759 0.29214022 0.54382466]
Matriz A:
 [[-0.29701456 -0.26439423 -0.15679793]
 [-0.6451581   0.14999985 -0.67940333]
 [-0.03124445 -0.04875874 -0.1536913 ]]

 Matriz de Hessenberg H:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

 Matriz de Krylov V (ortogonal):
 [[ 0.07693049  0.48024148  0.78670759]
 [-0.99337173  0.45140625  0.39432458]
 [-0.08540666  0.75206417 -0.47497294]]

Verificando que V es ortogonal...
¿Es V ortogonal? No :c


C:\Users\56940\AppData\Local\Temp\ipykernel_12708\3576768577.py:42: RuntimeWarning: invalid value encountered in divide
  u_s = u_s / norm_u_s


In [25]:
def householder_reflection_vector(x):
    """
    Calcula el vector de Householder 'w' para un vector completo 'x'
    tal que Px = (I - 2ww^T)x sea paralelo a e_1.
    """
    n = len(x)
    e1 = np.zeros(n)
    e1[0] = 1.0
    s = np.copysign(np.linalg.norm(x), x[0])
    u = x + s * e1
    norm_u = np.linalg.norm(u)
    return u / norm_u

def arnoldi_saad(A,b):
    n = A.shape[0]
    V, H = np.zeros((n,n)), np.zeros((n,n))

    # 1). Select a non zero v
    v1 = b / np.linalg.norm(b)
    z = v1

    householder_reflectors = []
    # 2). Main Loop
    for i in range(n):
        print("Iteración", i)
        # 3). Compute the Householder unit vector wj such that
        # w_j[i] = 0, for all i [1,...j-1]
        # ||w_j|| = 1
        # (Pj zj)[i] = 0, for all i [j+1,...,n]
        # Where Pj = I - 2 w_j w_j.T
        wj = householder_reflection_vector(z)
        Pj = np.eye(n) - 2 * np.outer(wj, wj)
        if i==0:
            print("primer vector wj:\n",wj)
        householder_reflectors.append(wj)
        # 4.) h_{j-1} = Pjej
        ej = np.zeros(n)
        ej[i] = 1.0
        H[:, i] = Pj @ ej

        # 5). vj = Pj ... P1 ej
        ej = np.zeros(n)
        ej[i] = 1.0
        for P in reversed(householder_reflectors):
            ej = P @ ej
        v = ej
        V[:, i] = ej

        if i < n:
            z = A @ v
            for P in householder_reflectors:
                z = P @ z
                   
    return V, H


N = 3
kappa_deseada = 10
A, x_true, b, kappa_emp = generate_system(N, kappa=kappa_deseada, structure="general", seed=42)

V, H = arnoldi_saad(A, b)

print("Matriz A:\n", A)
print("\n Matriz de Hessenberg H:\n", H)
print("\n Matriz de Krylov V (ortogonal):\n", V)


# random_vec_5 = np.ones(5)
# u5 = get_householder_vector(random_vec_5,1)
# print("print u5:", random_vec_5)
# print("house holder vec",u5)
# print("Householder debería eliminar a partir del elmento 1+1:\n", apply_householder(u5, random_vec_5))


print("\nVerificando que V es ortogonal...")
ortogonal = np.allclose(V.T @ V, np.eye(N))
print(f"¿Es V ortogonal? {'Sí :D' if ortogonal else 'No :c'}")

# print("\nVerificando la relación AV = VH...")
# AV = A @ V
# VH = V @ H

# print("\nResultado de A @ V:\n", AV)
# print("\nResultado de V @ H:\n", VH)

# are_close = np.allclose(AV, VH)
# print(f"\n¿Son AV y VH suficientemente cercanos? {'Sí :D' if are_close else 'No :c'}")

Iteración 0
primer vector wj:
 [ 0.73380191 -0.67686641 -0.05819463]


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [20]:
def householder_reflection_vector(x):
    """
    Calcula el vector de Householder 'w' para un vector completo 'x'
    tal que Px = (I - 2ww^T)x sea paralelo a e_1.
    """
    n = len(x)
    if n == 0 or np.linalg.norm(x) < 1e-12:
        return np.zeros(n)

    e1 = np.zeros(n)
    e1[0] = 1.0

    # norma de x con el signo de x[0]
    s = np.copysign(np.linalg.norm(x), x[0])
    # ahora la suma es de 2 nums con el mismo signo
    u = x + s * e1

    norm_u = np.linalg.norm(u)
    if norm_u < 1e-12:
        return np.zeros(n)
    return u / norm_u

def arnoldi_householder_gemini(A, b):
    """
    Implementación del método de Arnoldi con Householder según el
    enfoque de Y. Saad. (iterative methods for sparse linear systems)
    """
    n = A.shape[0]

    # --- 1. Transformación Inicial del Sistema ---
    # Encontrar el reflector P0 tal que P0 @ b = beta * e_1
    w0 = householder_reflection_vector(b)
    P0 = np.eye(n) - 2 * np.outer(w0, w0)

    # Transformar la matriz A
    A_bar = P0 @ A @ P0.T
    # A_bar = A

    # --- 2. Bucle de Arnoldi sobre el sistema transformado ---
    H = np.zeros((n, n))
    V_bar = np.zeros((n, n))    
    arnoldi_reflectors = []
    
    # El primer vector de la base para el sistema transformado es e_1
    v = np.zeros(n)
    v[0] = 1.0
    
    for j in range(n):
        # Almacenar el vector base v_j
        V_bar[:, j] = v  


        # z_{j+1} =  (P_1, ..., P_j) A v_j
        z = A_bar @ v
        for P in arnoldi_reflectors:
            z = P @ z
        
        # Llenar la columna j de H con los elementos conocidos de z
        H[:j+1, j] = z[:j+1]

        # Solo creamos un nuevo reflector si no es la última iteración
        if j < n - 1:
            # Construir el reflector P_{j+1} para anular la "cola" de z
            z_tail = z[j+1:].copy()
            w_tail = householder_reflection_vector(z_tail)
            
            w = np.zeros(n)
            w[j+1:] = w_tail
            Pj_plus_1 = np.eye(n) - 2 * np.outer(w, w)
            arnoldi_reflectors.append(Pj_plus_1)

            # El elemento subdiagonal h_{j+1, j} se revela tras aplicar P_{j+1}
            z_reflected = Pj_plus_1 @ z
            H[j+1, j] = z_reflected[j+1]

            # Calcular el SIGUIENTE vector base v_{j+1}
            next_v = np.zeros(n)
            next_v[j+1] = 1.0  # Empezar con el vector canónico e_{j+2}

            # Aplicar P_1, ..., P_{j+1} en orden inverso
            for i in range(len(arnoldi_reflectors) - 1, -1, -1):
                next_v = arnoldi_reflectors[i] @ next_v
            v = next_v
            
    # --- 3. Devolver Resultados para el Sistema Original ---
    # La base V para el problema original es P0 @ V_bar
    V_orig = P0 @ V_bar
    return V_orig, H


N = 3
kappa_deseada = 10
A, x_true, b, kappa_emp = generate_system(N, kappa=kappa_deseada, structure="general", seed=42)

V, H = arnoldi_householder_gemini(A, b)

print("Matriz A:\n", A)
print("\n Matriz de Hessenberg H:\n", H)
print("\n Matriz de Krylov V (ortogonal):\n", V)

print("\nVerificando que V es ortogonal...")
ortogonal = np.allclose(V.T @ V, np.eye(N))
print(f"¿Es V ortogonal? {'Sí :D' if ortogonal else 'No :c'}")

print("\nVerificando la relación AV = VH...")
AV = A @ V
VH = V @ H

print("\nResultado de A @ V:\n", AV)
print("\nResultado de V @ H:\n", VH)

are_close = np.allclose(AV, VH)
print(f"\n¿Son AV y VH suficientemente cercanos? {'Sí :D' if are_close else 'No :c'}")

Matriz A:
 [[-0.29701456 -0.26439423 -0.15679793]
 [-0.6451581   0.14999985 -0.67940333]
 [-0.03124445 -0.04875874 -0.1536913 ]]

 Matriz de Hessenberg H:
 [[ 0.15410541  0.77948821  0.48798673]
 [ 0.2522402  -0.38906978 -0.09911786]
 [ 0.          0.10663899 -0.06574164]]

 Matriz de Krylov V (ortogonal):
 [[-0.07693049 -0.95674068  0.28058684]
 [ 0.99337173 -0.04944331  0.10376877]
 [ 0.08540666 -0.28671002 -0.95420284]]

Verificando que V es ortogonal...
¿Es V ortogonal? Sí :D

Verificando la relación AV = VH...

Resultado de A @ V:
 [[-0.25318387  0.34219397  0.03884279]
 [ 0.14061237  0.80462425  0.48283101]
 [-0.05915816  0.07636846  0.13282626]]

Resultado de V @ H:
 [[-0.25318387  0.34219397  0.03884279]
 [ 0.14061237  0.80462425  0.48283101]
 [-0.05915816  0.07636846  0.13282626]]

¿Son AV y VH suficientemente cercanos? Sí :D


In [38]:
# triangular superior
N = 3
A = np.random.rand(N, N)
R = np.triu(A)
print(R)
R_inverse = np.linalg.inv(R).T
R_inverse_inverse = np.linalg.inv(R_inverse)
# print(R_inverse)
print(R_inverse_inverse.T - R)

[[0.1682394  0.41509951 0.82489397]
 [0.         0.68033677 0.73034947]
 [0.         0.         0.43201342]]
[[ 0.00000000e+00 -1.11022302e-16 -1.11022302e-16]
 [ 0.00000000e+00 -1.11022302e-16 -1.11022302e-16]
 [ 0.00000000e+00 -4.46787163e-17  0.00000000e+00]]
